In [1]:
import sys
sys.path.append('../')

from Datasets.BaseballDataset import BaseballDataset

import torch
import torch.nn as nn
import torch.optim as optim
import math
import torch.nn.functional as F
from torch.utils.data import DataLoader
import json
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

In [2]:
data_config_path = "../data/config.json"
valid_path = "../data/statcast_2015-2023_cleaned.csv"
sequence_length = 200
valid_data = pd.read_csv(valid_path)


In [27]:
valid_dataset = BaseballDataset(valid_data,data_config_path,sequence_length)

KeyboardInterrupt: 

In [15]:
len(valid_dataset)

18094

In [16]:
valid_dataset.continuous_label_indices

tensor([ 0,  8,  9, 17])

In [17]:
valid_dataset.categorical_label_indices

[tensor([21, 22, 23, 24, 25, 26, 27, 28, 29, 30]),
 tensor([52, 53, 54, 55, 56, 57, 58, 59, 60, 61])]

In [30]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Arguments:
            x: Tensor, shape ``[batch_size, seq_len, embedding_dim]``
        """
        x = x + self.pe[:x.size(1)].transpose(0, 1)
        return self.dropout(x)

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads, num_encoder_layers, hidden_dim, output_dim, sequence_length, dropout=0.1):
        super(TransformerModel, self).__init__()
        
        self.input_dim = input_dim
        self.sequence_length = sequence_length
        
        self.embedding = nn.Linear(input_dim, hidden_dim)
        self.positional_encoding = PositionalEncoding(hidden_dim, dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers)
        
        self.fc_layers = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)
        x = x[:, -1, :]  # Use the output of the last pitch in the sequence
        x = self.fc_layers(x)
        return x


def load_model(model_path, config_path):
    with open(config_path, 'r') as file:
        config = json.load(file)

    model = TransformerModel(
        input_dim=config['input_dim'],
        num_heads=config['num_heads'],
        num_encoder_layers=config['num_encoder_layers'],
        hidden_dim=config['hidden_dim'],
        output_dim=config['output_dim'],
        sequence_length=config['sequence_length'],
        dropout=config.get('dropout', 0.1)  # Optional: provide a default value for dropout if not in config
    )

    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set the model to evaluation mode
    return model



def make_preds(model, dataset, scaler_path, device, batch_size):

    #get column names in correct order
    flat_cat_names = []
    for names in dataset.categorical_label_names:
        flat_cat_names = flat_cat_names + names 
    col_names = dataset.continuous_label_names + flat_cat_names

    #create dataloader for dataset
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    model.eval()

    preds_array = [] #keep trask of preds for each batch
    true_array = [] #keep track of true values
    with torch.no_grad():
        idx = 0
        for sequence_tensor, cont_target_tensor, cat_target_tensor in loader:
            idx += 1
            if idx % 10 == 0:
                print(f"Starting Batch: {idx}")
                
            sequence_tensor, cont_target_tensor = sequence_tensor.to(device), cont_target_tensor.to(device)
            cat_targets = [t.to(device) for t in cat_target_tensor]
            output = model(sequence_tensor)

            #first k logits correspond to continuous outputs, k = cont_target.size(1)
            cont_output = output[:, :cont_target_tensor.size(1)].cpu().squeeze(0).detach().numpy()
            cont_targets = cont_target_tensor.cpu().squeeze(0).detach().numpy()

            #can have multiple kinds of categorical outputs. If cat_targets is (batch_size, 2, 10), there are 2 kinds of cateogorical outputs, each with 10 values.
            #The first 10 logits after the continuous logits will correspond to first categorical output, second 10 to the second, so this requires multiple softmaxes
            cat_probs = []
            cat_target_probs = []
            start_idx = cont_target_tensor.size(1)
            for cat_target in cat_targets:
                end_idx = start_idx + cat_target.size(1)
                cat_probs.append(nn.functional.softmax(output[:, start_idx:end_idx],dim=1).cpu().squeeze(0).detach().numpy())
                cat_target_probs.append(cat_target.cpu().squeeze(0).detach().numpy())
                start_idx = end_idx
    
            #cat continuous and categorical outputs together
            preds = cont_output
            for probs in cat_probs:
                preds = np.concatenate((preds, probs),axis=1)
            
            preds_array.append(preds)

            true = cont_targets
            for probs in cat_target_probs:
                true = np.concatenate((true, probs),axis=1)
            
            true_array.append(true)

    #make single preds pd     
    preds_array = np.vstack(preds_array)
    preds_pd = pd.DataFrame(preds_array, columns=col_names)

    true_array = np.vstack(true_array)
    true_pd = pd.DataFrame(true_array, columns=col_names)

    #scale continuous outputs back to real values
    with open(scaler_path, "rb") as file:
        scalers = pickle.load(file)

    for column, scaler in scalers.items():
        if column in preds_pd:
            preds_pd[column] = (preds_pd[column] * scaler.scale_) + scaler.mean_
            true_pd[column] = (true_pd[column] * scaler.scale_) + scaler.mean_
    

    return preds_pd, true_pd

In [25]:
med200 = load_model("med_200_output_updated/transformer_model.pth", "med_200_output_updated/model_config.json")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
med200.to(device)

cuda


TransformerModel(
  (embedding): Linear(in_features=75, out_features=42, bias=True)
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=42, out_features=42, bias=True)
        )
        (linear1): Linear(in_features=42, out_features=42, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=42, out_features=42, bias=True)
        (norm1): LayerNorm((42,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((42,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=42, out_features=42, bias=True)
    (1): ReLU()
    (2): L

In [31]:
med200preds, true = make_preds(med200,valid_dataset,"../data/statcast_2023-2024_cleaned_scalers.pkl",device, batch_size=200)

Starting Batch: 10
Starting Batch: 20
Starting Batch: 30
Starting Batch: 40
Starting Batch: 50
Starting Batch: 60
Starting Batch: 70
Starting Batch: 80
Starting Batch: 90


c:\Users\declan\anaconda3\envs\pytorchCUDA\lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.5.1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [32]:
med200preds

,launch_speed,hc_x,hc_y,launch_angle,events_B,events_S,events_double,events_field_out,events_hit_by_pitch,events_home_run,...,hit_location_0.0,hit_location_1.0,hit_location_2.0,hit_location_3.0,hit_location_4.0,hit_location_5.0,hit_location_6.0,hit_location_7.0,hit_location_8.0,hit_location_9.0
0,54.046708,54.049954,54.044296,54.049585,0.342297,0.411488,0.011767,0.10972,0.002719,0.009172,...,0.79013,0.006635,0.058353,0.011548,0.018041,0.016361,0.021786,0.023337,0.027624,0.026186
1,54.046708,54.049954,54.044296,54.049585,0.342297,0.411488,0.011767,0.10972,0.002719,0.009172,...,0.79013,0.006635,0.058353,0.011548,0.018041,0.016361,0.021786,0.023337,0.027624,0.026186
2,54.046708,54.049954,54.044296,54.049585,0.342297,0.411488,0.011767,0.10972,0.002719,0.009172,...,0.79013,0.006635,0.058353,0.011548,0.018041,0.016361,0.021786,0.023337,0.027624,0.026186
3,54.046708,54.049954,54.044296,54.049585,0.342297,0.411488,0.011767,0.10972,0.002719,0.009172,...,0.79013,0.006635,0.058353,0.011548,0.018041,0.016361,0.021786,0.023337,0.027624,0.026186
4,54.046708,54.049954,54.044296,54.049585,0.342297,0.411488,0.011767,0.10972,0.002719,0.009172,...,0.79013,0.006635,0.058353,0.011548,0.018041,0.016361,0.021786,0.023337,0.027624,0.026186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18089,54.046708,54.049954,54.044296,54.049585,0.342297,0.411488,0.011767,0.10972,0.002719,0.009172,...,0.79013,0.006635,0.058353,0.011548,0.018041,0.016361,0.021786,0.023337,0.027624,0.026186
18090,54.046708,54.049954,54.044296,54.049585,0.342297,0.411488,0.011767,0.10972,0.002719,0.009172,...,0.79013,0.006635,0.058353,0.011548,0.018041,0.016361,0.021786,0.023337,0.027624,0.026186
18091,54.046708,54.049954,54.044296,54.049585,0.342297,0.411488,0.011767,0.10972,0.002719,0.009172,...,0.79013,0.006635,0.058353,0.011548,0.018041,0.016361,0.021786,0.023337,0.027624,0.026186
18092,54.046708,54.049954,54.044296,54.049585,0.342297,0.411488,0.011767,0.10972,0.002719,0.009172,...,0.79013,0.006635,0.058353,0.011548,0.018041,0.016361,0.021786,0.023337,0.027624,0.026186


In [33]:
true

,launch_speed,hc_x,hc_y,launch_angle,events_B,events_S,events_double,events_field_out,events_hit_by_pitch,events_home_run,...,hit_location_0.0,hit_location_1.0,hit_location_2.0,hit_location_3.0,hit_location_4.0,hit_location_5.0,hit_location_6.0,hit_location_7.0,hit_location_8.0,hit_location_9.0
0,53.767208,53.853561,53.855617,53.776220,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,53.767208,53.853561,53.855617,53.776220,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,54.964541,54.805330,55.426675,54.522994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,53.767208,53.853561,53.855617,53.776220,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,53.767208,53.853561,53.855617,53.776220,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18089,55.059068,54.948205,55.284953,54.467677,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
18090,53.767208,53.853561,53.855617,53.776220,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18091,54.696717,53.853561,53.855617,54.900991,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18092,54.678539,53.853561,53.855617,54.716602,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
med200preds['hit_location_0.0'].unique()

array([0.7901304], dtype=float32)